<a href="https://colab.research.google.com/github/jdasam/mas1004-2022/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Image Classification
- In this assignment you have to train image classification model with __your own data__
- You have to submit 1) a report, 2) code and 3) data
  1. Report in free format (submit in PDF)
    - 1) Introduce why you selected these four categories
    - 2) Describe the very first result of training the model, especially about the error cases
    - 3) Explain your criteria for data cleaning
    - Your submission would be evalulated mainly with the report
    - Please include the plot figures that were generated during the experiment in your report
    - DO NOT INCLUDE MORE THAN FIVE Experiments in your report
  2. code (submit in ipynb)
  3. data (submit in zip files)

- Evaluation Criteria:
  1. How well did you apply the knowledge from the class
  2. How well did you improve the result
  3. How well did you analyze the result

## 0. Install Library and Import

In [ ]:
!pip install -q jmd_imagescraper

import random
import shutil
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time


from pathlib import Path
from jmd_imagescraper.core import duckduckgo_search
from PIL import Image



## 1. Configure Settings
- 1) Select the storage type
  - Since Colab storage is volatile, your image dataset or model would disappear after the session ends.
  - There are two options to save the data permanently:
    1. Save your data in your local computer by downloading it from Colab. select `SAVE_TYP='local'`
    2. Save your data in your Google Drive storage. select `SAVE_TYP='google_drive'`
  - If you are running on your local computer, use `SAVE_TYP='local'`
- 2) Select Number of images you want to collect
  - `NUM_IMG` will define the number of image per category for crawling
  - `100<=NUM_IMG<=500` of your choice

In [ ]:
# SAVE_TYP = 'local'
SAVE_TYP = 'google_drive'
NUM_IMG = 100 # This will define how many images you will download

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  if SAVE_TYP == 'local': # if you selected to save it in local
    from google.colab import files
  else:
    from google.colab import drive # if you selected to save it in your Google Drive
    drive.mount('/content/drive')
    GGL_DIR = Path('/content/drive/MyDrive/Data_AI_Assignment2/')
    GGL_DIR.mkdir(exist_ok=True)

IMG_DIR = Path('images')
if IN_COLAB and SAVE_TYP=="google_drive":
  PLOT_DIR = GGL_DIR / "plots/"
else:
  PLOT_DIR = Path('plots')
PLOT_DIR.mkdir(exist_ok=True)
DEL_DIR = IMG_DIR / 'deleted'

## Problem 1: Collect Data (20 pts)
- You have to select 4 image categories of your own choice
- The selected categories will be used as an image class for the classification model
- Evaluation:
    - If you select four categories that does not make error on the code, you will get 10 pts
    - If the selected 4 categories and collected images are reasonable for training an image classification model, you will get another 5 pts
    - Remaining 5 pts will be evaluated based on how interesting is your choice

In [ ]:
def configure_image_categories():
  '''
  input: None
  output: List of strings that contains 4 classes

  example: image_keywords = ['football', 'basketball', 'baseball', 'volleyball']
  '''
  # TODO: Complete the function
  image_keywords =  ['football', 'basketball', 'baseball', 'volleyball']
  return image_keywords

## Image Crawling
- Following code will automatically crawl the images with the keyword you have choosen
- After running it, it will automatically split train and valid to ratio of 8:2
  - Training data will be located `images/train`
  - Validation data will be locatd `images/valid`


#### Pre-defined Functions (run it without opening it)

In [ ]:
'''
You don't have to change this cell
'''

def get_image_using_duckduckgo(img_dir:Path, image_types: list, num_img:int):
  for typ in image_types:
    duckduckgo_search(img_dir, typ, typ, max_results=NUM_IMG)
    typ_dir = img_dir / typ

def split_train_and_valid(img_dir, image_types, random_seed=0):
  random.seed(0)
  valid_indices = random.sample(range(NUM_IMG), NUM_IMG//5)
  image_keywords = [child.name for child in img_dir.iterdir() if child.is_dir()]
  for typ in image_keywords:
    typ_dir = img_dir / typ
    train_dir = img_dir / 'train' / typ
    test_dir = img_dir / 'test' / typ
    train_dir.mkdir(parents=True, exist_ok=True)
    test_dir.mkdir(parents=True, exist_ok=True)
    img_files = list(typ_dir.rglob('*.jpg'))
    valid_imgs = [img_files[i] for i in valid_indices]
    for fn in valid_imgs:
      shutil.move(fn, test_dir/fn.name)
    img_files = list(typ_dir.rglob('*.jpg'))
    for fn in img_files:
      shutil.move(fn, train_dir/fn.name)
    os.rmdir(typ_dir)

def save_fig_with_date(figname):
  plt.savefig(PLOT_DIR/f"{figname}_{'_'.join(time.ctime().split(' ')[1:4])}.png")

def file_name_with_date(filename):
  filename = Path(filename)
  return f"{filename.stem}_{'_'.join(time.ctime().split(' ')[1:4])}{filename.suffix}"

def save_file(fn):
  if IN_COLAB: # If you are running this notebook on Colab 
    if SAVE_TYP == 'local': # if you selected to save it in local
      files.download(fn)   # download the file to your local computer 
    else:
      shutil.copy(fn, GGL_DIR)  # copy the file to your google drive

def plot_random_sampled_images(img_dir, ncols=4, nrows=5,random_seed=0):
  list_of_image_files = list(img_dir.rglob("*.jpg"))
  random.seed(random_seed)
  random.shuffle(list_of_image_files)
  plt.figure(figsize=(ncols*5,nrows*4))
  for i in range(ncols*nrows):
    fname = list_of_image_files[i]
    image = Image.open(fname)
    plt.subplot(nrows,ncols, i+1)
    plt.imshow(image)
    plt.title('/'.join(str(fname.parent).split('/')[-2:]))
    plt.axis('off')
  save_fig_with_date("dataset_check") # save figure as png in PLOT_DIR

### Download or Unzip
- If this is your first time to download the dataset, it will automatically download the images fro duckduckgo search
- If you already ran the crawling code and have `image_data.zip` file in your local or google drive, you can upload it to the current Colab storage and unzip it

In [ ]:
image_types = configure_image_categories()
assert len(image_types)==4, "The length of image types has to be 4"
assert all( isinstance(typ, str) for typ in image_types), "Every element of image_types has to be string"


- If you want to delete every file in the `images/`, you can run following code
  - `!rm -rf images/`
  - It will forcely remove every file and directory in images. So please be careful when using it
  

In [ ]:
# delete the images/ folder if you want to re-compose your dataset

- Following code will download the dataset or unzip the pre-saved zip files

In [ ]:
if not IMG_DIR.exists(): # If there is no image directory
  if Path("image_data_cleaned.zip").exists(): # if there is already cleaned dataset
    print("Extracting image_data_cleaned.zip")
    os.system("unzip -q image_data_cleaned.zip") # unzip image_data_cleaned.zip
  elif IN_COLAB and SAVE_TYP == "google_drive" and (GGL_DIR/"image_data_cleaned.zip").exists(): # if there is already cleaned dataset on GD
    print("Extracting image_data_cleaned.zip from Google Drive")
    shutil.copy(str(GGL_DIR/"image_data_cleaned.zip"), "/content/") # copy it to colab storage
    os.system("unzip -q image_data_cleaned.zip") # and unzip it
  elif Path("image_data.zip").exists(): # If image_data.zip file exists
    print("Extracting image_data.zip")
    os.system("unzip -q image_data.zip") # Unzip the zip file
  elif IN_COLAB and SAVE_TYP == "google_drive" and (GGL_DIR/"image_data.zip").exists():
    print("Extracting image_data.zip from Google Drive")
    shutil.copy(str(GGL_DIR/"image_data.zip"), "/content/")
    os.system("unzip -q image_data.zip")
  else: # If there is no image directory and also no image_data.zip
    print("Downloading images from DuckDuckGo")
    get_image_using_duckduckgo(IMG_DIR, image_types, NUM_IMG) # Download image files
    split_train_and_valid(IMG_DIR, image_types) # Split train and valid into different directories
    os.system(f"zip -rq image_data.zip {IMG_DIR}") # Make zip file that contains images directory
    save_file("image_data.zip")
else:
  print(f"IMG_DIR ({IMG_DIR}) already exists")

### Plot your Dataset
- It is always important to check your dataset in detail
- Run the script below and see the result
  - The figure will automatically saved as png with current time to `PLOT_DIR`

In [ ]:
plot_random_sampled_images(IMG_DIR, ncols=4, nrows=5, random_seed=0)

## Problem 2: Report the first training result (20 pts)
- You don't have to change the code for Problem 2 if you have made the correct dataset on Problem 1
- Evaluation: How well did you explain the result of your first training on Report

### DataBlock
Class that can feed data in a right format to fastai model.
* *blocks*: type of input/labels. 
  * *(ImageBlock, CategoryBlock)* means that input is image and label is categorical value
* *get_items*: a function to load item from storage. 
  * *get_image_files* is a fastai function that get lists of image files in a given directory
* *splitter*: a function to split train and validation
  * *GrandparentSplitter* splits dataset by its Grandparent directory (parent directory of parent directory) name.
* *get_y*: a function to get label 
  * *parent_label* means to use name of parent directory of item as the label
* *item_tfms*: item transformation. How to transform the loaded item before feeding it to a neural network model
  * *Resize* is to resize entire image into square of certain size. 

In [ ]:
class ImageSet:
  def __init__(self, path_dir, file_types=['jpg', 'png'], transform=None):
    self.path = Path(path_dir)
    self.image_fns = sorted(item for y in [list(self.path.rglob(f'*.{x}')) for x in file_types] for item in y) 
    self.classes = sorted(list(set([x.parent.name for x in self.image_fns])))
    self.cls2idx = {k: i for i, k in enumerate(self.classes)}
    self.transform = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
  
  def __len__(self):
    return len(self.image_fns)
  
  def __getitem__(self, idx):
    img_path = self.image_fns[idx]
    img = self.transform(Image.open(img_path).convert('RGB'))
    cls = img_path.parent.name
    return img, self.cls2idx[cls]

trainset = ImageSet(IMG_DIR/'train')
testset = ImageSet(IMG_DIR/'test')

Check the Batch result

In [ ]:
tensor2pil = transforms.Compose([
    transforms.Normalize(mean=[0, 0, 0], std=[4.3668, 4.4643, 4.4444]),
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    transforms.ToPILImage()
])


def show_batch(dataloader, ncols=4, nrows=5, random_seed=0):
  torch.manual_seed(random_seed)
  images, labels = next(iter(dataloader))
  plt.figure(figsize=(ncols*5,nrows*4))
  for i in range(ncols*nrows):
    plt.subplot(nrows, ncols, i+1)
    pil_img = tensor2pil(images[i])
    plt.imshow(pil_img)
    plt.title(trainset.classes[labels[i]])
    plt.axis('off')

train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=True)

In [ ]:
show_batch(train_loader, random_seed=0)
save_fig_with_date("dls_default_train_batch") # Don't forget to always save the data

### Train the first model

In [ ]:
import numpy as np
class Trainer:
  def __init__(self, model, train_loader, valid_loader):
    self.model = model
    self.train_loader = train_loader
    self.valid_loader = valid_loader
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.model.to(self.device)
    self.criterion = nn.NLLLoss()
    self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
    self.best_loss = np.inf
    self.best_acc = 0.0
    self.best_epoch = 0
    self.best_model = None
    self.train_losses = []
    self.valid_losses = []
    self.train_accs = []
    self.valid_accs = []

  def fine_tune(self, num_epochs):
    self.model.train()
    for epoch in range(num_epochs):
      for batch in self.train_loader:
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()


model = models.resnet18(pretrained=True)
trainer = Trainer(model, train_loader, test_loader)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate) # Define learn
learn.fine_tune(20) # This will run fine tuning
save_name =file_name_with_date("1st_learn.pkl") # 
learn.export(save_name) # save learn result as pkl file
save_file(save_name) # move file to local or google drive


#### Load Model
- You can load the pkl learn object by:
`learn = load_learner(save_name)`
- Since the `learn.export()` only export model parameters and optimizer parameters, it will lose `learn.recorder` object

In [ ]:
loaded_learner = load_learner(save_name)

### Check the train result

In [ ]:
learn.recorder.plot_loss()
save_fig_with_date("loss")

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
save_fig_with_date("confusion")
interp.plot_top_losses(10, ncols=1)
save_fig_with_date("top_losses")

In [ ]:
interp.print_classification_report()

In [ ]:
interp.most_confused()

## Problem 3: Clean your Dataset (30 Pts)
- The goal of data cleaning is to make sure that your dataset consists of images with specific characteristics you want.

- Cell below is the modified version of `jmd_imagescraper.imagecleaner import display_image_cleaner`
  - Once you run it, you can call `display_image_clenaer`
  - *YOU HAVE TO RELOAD* the UI when you make different `display_image_cleaner`
- If you click `delete` button, the image file will be moved to `DEL_DIR`, which is in default `images/deleted` 
- After the cleaning YOU HAVE TO SAVE YOUR RESULT by running the cell below.
- Evaluation Criteria:
  1. How did you define the rule to ommit certain examples


#### Cleaner code (run it without opening it)

In [ ]:
#hide
from pathlib import Path
from PIL import Image as PImage
from PIL import ImageDraw as PImageDraw
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO

##########################################################################################
# globals & event handlers
##########################################################################################
ICLN_BATCH_SZ = 8

# this may look nauseating and but creating new widgets is literally about 10x slower than 
# updating existing ones so the ui gets created once and updated forever more
icln_base_path = None
icln_folder = None
icln_batches = None
icln_pager = None
icln_grid = None
icln_empty_folder = None

def delete_on_click(btn):
  fn, img, batch, idx = btn.tag
  img.value = icln_deleted_img() # display red 'deleted' cross
  icln_batches[batch][idx] = ""  # so we know it's deleted as we page back & forth
  btn.disabled = True
  # try:    Path(fn).unlink()      # dbl-clicks result in us trying to delete twice
  try: shutil.move(str(fn), DEL_DIR)
  except Exception as e: print(e)

def paging_on_click(btn):
  folder, batch = btn.tag
  icln_render_batch(folder, batch)

def reload_on_click(btn):
  icln_render_batch(icln_folder, 0, force_reload=True)

def folder_on_change(change):
  if(change["type"] == "change" and change["name"] == "value"):
    icln_render_batch(change["new"], 0)
  
##########################################################################################
# UI creation
##########################################################################################
def icln_deleted_img():
  # creates the red "deleted" placeholder cross and caches it
  DELETED_IMG = "deleted_img"
  
  if(DELETED_IMG not in icln_deleted_img.__dict__):
    img = PImage.new("RGB",(150,150), color="white")

    draw = PImageDraw.Draw(img)
    draw.line((5, 5, 140, 140), fill="red", width=10)
    draw.line((5, 140, 140, 5), fill="red", width=10)
    
    bio = BytesIO()
    img.save(bio, 'JPEG')
    icln_deleted_img.__dict__[DELETED_IMG] = bio.getvalue()

  return icln_deleted_img.__dict__[DELETED_IMG]

def icln_create_widgets(batch_size):
  # create the UI widgets
  global icln_pager
  global icln_grid
  global icln_empty_folder

  # image/delete button pairs
  display_items = []
  for i in range(batch_size):
    img = widgets.Image()
    img.layout.width="150px"
    btn = widgets.Button(description="Delete")
    btn.on_click(delete_on_click)
    box = widgets.VBox(children=[img,btn])
    box.layout.margin = "5px"
    display_items.append(box)

  # paging
  btnFirst = widgets.Button(description="|<<") 
  btnPrev = widgets.Button(description="<<")
  lblPage = widgets.Label(value="Page NNN of KKK")
  lblPage.layout = widgets.Layout(display="flex", justify_content="center", width="100px")
  btnNext = widgets.Button(description=">>")
  btnLast = widgets.Button(description=">>|")
  
  pgbtns = [btnFirst, btnPrev, btnNext, btnLast]
  for btn in pgbtns: btn.on_click(paging_on_click)
  for btn in pgbtns: btn.layout.width = "60px"

  # folder drop down
  folders = [f.stem for f in icln_base_path.glob("*") if (f.is_dir() and f.stem[0] != ".")]
  folders.sort()
  rootfiles = [f for f in icln_base_path.glob("*.jpg") if f.is_file()]
  if(len(rootfiles) > 0): folders = ["/"] + folders
  ddlFolder = widgets.Dropdown(options=folders, description="Folder: ")
  ddlFolder.observe(folder_on_change)

  # reload button
  btnReload = widgets.Button(description="↻")
  btnReload.layout = widgets.Layout(width="40px", margin="0px 0px 0px 10px")
  btnReload.on_click(reload_on_click)

  # plug it all in and display
  icln_pager = widgets.HBox(children=[btnFirst, btnPrev, lblPage, btnNext, btnLast, 
                                      ddlFolder, btnReload])  
  icln_grid = widgets.GridBox(display_items, 
                              layout=widgets.Layout(grid_template_columns="repeat(4, 25%)",
                                                    margin="15px"))
  icln_empty_folder = widgets.HTML(value="<h2>No images left to display in this folder.</h2>")
  icln_empty_folder.layout.visibility = "hidden"

  display(icln_pager)
  display(icln_empty_folder)
  display(icln_grid)
  
##########################################################################################
# UI rendering
##########################################################################################
def icln_render_batch(folder, batch, force_reload=False):
  global icln_folder
  global icln_batches
  global icln_pager
  global icln_grid

  if(folder == "/"): folder = ""
  path = icln_base_path/folder

  if((icln_folder != folder) or (force_reload)): 
    # get the files, split into batches  
    files = list(path.glob("*.jpg"))
    icln_batches = [files[i:i + ICLN_BATCH_SZ] for i in range(0, len(files), ICLN_BATCH_SZ)]
    icln_folder = folder

    if(len(files) == 0):
      # fail gracefully if they've deleted every image in this folder
      icln_empty_folder.layout.visibility = "visible"
      # icln_grid.layout.visibility = "hidden" <-- doesn't work :-@
      for child in icln_grid.children: child.layout.visibility = "hidden"
      btnFirst, btnPrev, lblPage, btnNext, btnLast,_,_ = icln_pager.children
      lblPage.value = "Page 0 of 0"
      for btn in [btnFirst, btnPrev, btnNext, btnLast]: btn.disabled = True
      return
    else:
      icln_empty_folder.layout.visibility = "hidden"
      icln_grid.layout.visibility = "visible"

  # display the images
  for i, fp in enumerate(icln_batches[batch]):
    icln_grid.children[i].layout.visibility = "visible"
    img = icln_grid.children[i].children[0]
    btn = icln_grid.children[i].children[1]

    if(fp == ""):
      img.value = icln_deleted_img()
      btn.disabled = True
    else:
      img.value = open(fp, "rb").read()
      btn.tag = (fp, img, batch, i)
      btn.disabled = False

  if(len(icln_batches[batch]) < ICLN_BATCH_SZ):
    # partial batch on the last page, hide the rest of the grid
    for i in range(len(icln_batches[batch]), ICLN_BATCH_SZ):
      icln_grid.children[i].layout.visibility = "hidden"
    
  # update the paging controls
  btnFirst, btnPrev, lblPage, btnNext, btnLast,_,_ = icln_pager.children
  btnFirst.tag = (folder, 0) 
  btnPrev.tag = (folder, max(0, batch-1)) 
  btnNext.tag = (folder, min(len(icln_batches)-1, batch+1)) 
  btnLast.tag = (folder, len(icln_batches)-1) 
  lblPage.value = "Page {} of {}".format(batch+1, len(icln_batches))
  for btn in [btnFirst, btnPrev, btnNext, btnLast]: btn.disabled = btn.tag[1] == batch

def display_image_cleaner(path):
    '''Display the image cleaner widget for the given folder'''
    global icln_base_path; icln_base_path = Path(path)
    
    icln_create_widgets(ICLN_BATCH_SZ)
    _,_,_,_,_,ddlFolder,_ = icln_pager.children
    icln_render_batch(ddlFolder.value, 0)

#### Run `display_image_cleaner`

In [ ]:
DEL_DIR.mkdir(exist_ok=True, parents=True)
display_image_cleaner('images/train')

In [ ]:
# DO NOT FORGET TO CLICK RELOAD BUTTON on the top right of the UI 
display_image_cleaner('images/valid') 

- Don't Forget to save your data

In [ ]:
os.system(f"zip -rq image_data_cleaned.zip {IMG_DIR}") # Make zip file that contains images directory
save_file("image_data_cleaned.zip")